<a href="https://colab.research.google.com/github/ChristianAgyapong/Medgemma/blob/main/notebooks/quick_start_with_hugging_face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

~~~
Copyright 2025 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
~~~

# Quick start with Hugging Face

<table><tbody><tr>
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/google-health/medgemma/blob/main/notebooks/quick_start_with_hugging_face.ipynb">
      <img alt="Google Colab logo" src="https://www.tensorflow.org/images/colab_logo_32px.png" width="32px"><br> Run in Google Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fgoogle-health%2Fmedgemma%2Fmain%2Fnotebooks%2Fquick_start_with_hugging_face.ipynb">
      <img alt="Google Cloud Colab Enterprise logo" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" width="32px"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/google-health/medgemma/blob/main/notebooks/quick_start_with_hugging_face.ipynb">
      <img alt="GitHub logo" src="https://github.githubassets.com/assets/GitHub-Mark-ea2971cee799.png" width="32px"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://huggingface.co/collections/google/medgemma-release-680aade845f90bec6a3f60c4">
      <img alt="Hugging Face logo" src="https://huggingface.co/front/assets/huggingface_logo-noborder.svg" width="32px"><br> View on Hugging Face
    </a>
  </td>
</tr></tbody></table>

This notebook provides a basic demo of using MedGemma, a collection of Gemma 3 variants that are trained for performance on medical text and image comprehension. MedGemma is intended to accelerate building healthcare-based AI applications.

Learn more about the model at the [HAI-DEF developer site](https://developers.google.com/health-ai-developer-foundations/medgemma).

## Setup

To complete this tutorial, you'll need to have a runtime with [sufficient resources](https://ai.google.dev/gemma/docs/core#sizes) to run the MedGemma model.

You can try out MedGemma 4B for free in Google Colab using a T4 GPU:

1. In the upper-right of the Colab window, select **▾ (Additional connection options)**.
2. Select **Change runtime type**.
3. Under **Hardware accelerator**, select **T4 GPU**.

**Note**: To run the demo with MedGemma 27B in Google Colab, you will need a runtime with an A100 GPU and use 4-bit quantization to reduce memory usage. The performance of quantized versions has not been evaluated.

### Get access to MedGemma

Before you get started, make sure that you have access to MedGemma models on Hugging Face:

1. If you don't already have a Hugging Face account, you can create one for free by clicking [here](https://huggingface.co/join).
2. Head over to the [MedGemma model page](https://huggingface.co/google/medgemma-4b-it) and accept the usage conditions.

### Authenticate with Hugging Face

Generate a Hugging Face `read` access token by going to [settings](https://huggingface.co/settings/tokens).

If you are using Google Colab, add your access token to the Colab Secrets manager to securely store it. If not, proceed to run the cell below to authenticate with Hugging Face.

1. Open your Google Colab notebook and click on the 🔑 Secrets tab in the left panel. <img src="https://storage.googleapis.com/generativeai-downloads/images/secrets.jpg" alt="The Secrets tab is found on the left panel." width=50%>
2. Create a new secret with the name `HF_TOKEN`.
3. Copy/paste your token key into the Value input box of `HF_TOKEN`.
4. Toggle the button on the left to allow notebook access to the secret.

In [24]:
! pip install -U transformers

In [25]:
from transformers import pipeline
from PIL import Image
import requests
import torch
import gc

# Clear GPU memory before loading pipeline
if 'pipe' in globals():
    del pipe
gc.collect()
torch.cuda.empty_cache()

# Initialize the MedGemma pipeline
pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-4b-it",
    torch_dtype=torch.bfloat16,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

# Load image
image_url = "https://i.ibb.co/B5MrLM9V/EC-38.jpg"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

# Create message with image only
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are an expert dermatologist, diagnose based on te image."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image}
        ]
    }
]

# Generate output
output = pipe(messages, max_new_tokens=200)

# Print result
print(output[0]["generated_text"][-1]["content"])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


Based on the image, the most likely diagnosis is **Tinea Pedis (Athlete's Foot)**.

Here's why:

*   **Appearance:** The image shows a scaly, erythematous (red) patch with some scaling and possibly some mild inflammation. This is a classic presentation of tinea pedis.
*   **Location:** The lesion is located on the foot, which is a common site for tinea pedis.
*   **Symptoms:** While not explicitly mentioned, tinea pedis often presents with itching, burning, and scaling.

**Important Considerations:**

*   **Differential Diagnosis:** Other possibilities to consider include:
    *   **Eczema:** Eczema can also present with scaling and redness, but it often has a more pruritic (itchy) nature and may be more widespread.
    *   **Psoriasis:** Psoriasis can also cause scaling and redness, but it often has a more silvery


In [26]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()


In [27]:
from transformers import pipeline
from PIL import Image
import requests
import torch
import gc

# Clear GPU memory before loading pipeline
if 'pipe' in globals():
    del pipe
gc.collect()
torch.cuda.empty_cache()

# Initialize the MedGemma pipeline
pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-4b-it",
    torch_dtype=torch.bfloat16,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

# Load image
image_url = "https://i.ibb.co/gbMFhcRt/EC-41.jpg"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

# Create message with image only
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are an expert dermatologist, diagnose based on te image."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image}
        ]
    }
]

# Generate output
output = pipe(messages, max_new_tokens=200)

# Print result
print(output[0]["generated_text"][-1]["content"])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


Based on the image, the most likely diagnosis is **post-inflammatory hyperpigmentation (PIH)**.

Here's why:

*   **Hyperpigmentation:** The skin tone appears darker than the surrounding skin.
*   **Possible Cause:** PIH is a common condition that occurs after inflammation of the skin, such as from acne, eczema, or other skin conditions.

**Important Considerations:**

*   **Further Evaluation:** While PIH is the most probable diagnosis, a definitive diagnosis requires a clinical examination by a dermatologist.
*   **Other Possibilities:** Other possibilities, though less likely based on the image alone, could include:
    *   **Melasma:** This is a more common cause of hyperpigmentation, but it typically presents as more symmetrical patches.
    *   **Sunspots (Solar Lentigines):** These are flat, brown spots that develop from sun exposure.
    *   **Other skin conditions:** Certain skin


In [1]:
from transformers import pipeline
from PIL import Image
import requests
import torch
import gc

# Clear GPU memory before loading pipeline
if 'pipe' in globals():
    del pipe
gc.collect()
torch.cuda.empty_cache()

# Initialize the MedGemma pipeline
pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-4b-it",
    torch_dtype=torch.bfloat16,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

# Load image
image_url = "https://i.ibb.co/gbMFhcRt/EC-461.jpg"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

# Create message with image only
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are an expert dermatologist, diagnose based on te image."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image}
        ]
    }
]

# Generate output
output = pipe(messages, max_new_tokens=200)

# Print result
print(output[0]["generated_text"][-1]["content"])

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cuda


Based on the image, the most likely diagnosis is **post-inflammatory hyperpigmentation (PIH)**.

Here's why:

*   **Hyperpigmentation:** The skin tone appears darker than the surrounding skin.
*   **Possible Cause:** PIH is a common condition that occurs after inflammation of the skin, such as from acne, eczema, or other skin conditions.

**Important Considerations:**

*   **Further Evaluation:** While PIH is the most probable diagnosis, a definitive diagnosis requires a clinical examination by a dermatologist.
*   **Other Possibilities:** Other possibilities, though less likely based on the image alone, could include:
    *   **Melasma:** This is a more common cause of hyperpigmentation, but it typically presents as more symmetrical patches.
    *   **Sunspots (Solar Lentigines):** These are flat, brown spots that develop from sun exposure.
    *   **Other skin conditions:** Certain skin


In [2]:
from transformers import pipeline
from PIL import Image
import requests
import torch
import gc

# Clear GPU memory before loading pipeline
if 'pipe' in globals():
    del pipe
gc.collect()
torch.cuda.empty_cache()

# Initialize the MedGemma pipeline
pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-4b-it",
    torch_dtype=torch.bfloat16,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

# Load image
image_url = "https://i.ibb.co/WWBGDx3Y/EC-29.jpg"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

# Create message with image only
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are an expert dermatologist, diagnose based on te image."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image}
        ]
    }
]

# Generate output
output = pipe(messages, max_new_tokens=200)

# Print result
print(output[0]["generated_text"][-1]["content"])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


Based on the image, the most likely diagnosis is **post-inflammatory hyperpigmentation (PIH)**.

Here's why:

*   **Hyperpigmentation:** The skin tone is darker than the surrounding skin, indicating an increase in melanin production.
*   **Possible Cause:** The image shows a skin lesion with a darker area. This could be due to a previous inflammatory skin condition, such as eczema, psoriasis, or acne. The darker area represents the area of skin that has been affected by the inflammation.

**Important Considerations:**

*   **Further Evaluation:** A definitive diagnosis requires a clinical examination by a dermatologist. This includes assessing the lesion's size, shape, borders, color, and texture.
*   **Other Possibilities:** While PIH is the most likely diagnosis, other possibilities include:
    *   **Melasma:** This is a common skin condition that causes brown or gray-brown patches, usually on the face.
    


In [3]:
from transformers import pipeline
from PIL import Image
import requests
import torch
import gc

# Clear GPU memory before loading pipeline
if 'pipe' in globals():
    del pipe
gc.collect()
torch.cuda.empty_cache()

# Initialize the MedGemma pipeline
pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-4b-it",
    torch_dtype=torch.bfloat16,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

# Load image
image_url = "https://i.ibb.co/zVVbHrKW/EC-15.jpg"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

# Create message with image only
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are an expert dermatologist, diagnose based on te image."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image}
        ]
    }
]

# Generate output
output = pipe(messages, max_new_tokens=200)

# Print result
print(output[0]["generated_text"][-1]["content"])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


Based on the image, the most likely diagnosis is **actinic keratosis (AK)**.

Here's why:

*   **Appearance:** The image shows rough, scaly patches on the skin, particularly on the palms. This is a classic presentation of AK.
*   **Location:** AKs often occur on sun-exposed areas, such as the hands.
*   **Risk Factors:** The patient's age and the presence of sun exposure increase the likelihood of AK.

**Important Considerations:**

*   This is a preliminary diagnosis based on a single image. A definitive diagnosis requires a clinical examination by a dermatologist, potentially including a biopsy.
*   Other conditions, such as psoriasis, eczema, or other skin cancers, could potentially present similarly.

**Next Steps:**

The patient should consult a dermatologist for a proper evaluation and diagnosis. The dermatologist will likely perform a physical examination, possibly a biopsy, and may order other tests to rule out other conditions


In [4]:
from transformers import pipeline
from PIL import Image
import requests
import torch
import gc

# Clear GPU memory before loading pipeline
if 'pipe' in globals():
    del pipe
gc.collect()
torch.cuda.empty_cache()

# Initialize the MedGemma pipeline
pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-4b-it",
    torch_dtype=torch.bfloat16,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

# Load image
image_url = "https://i.ibb.co/FLCSBs7Z/EC-12.jpg"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

# Create message with image only
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are an expert dermatologist, diagnose based on te image."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image}
        ]
    }
]

# Generate output
output = pipe(messages, max_new_tokens=200)

# Print result
print(output[0]["generated_text"][-1]["content"])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


Based on the image, the most likely diagnosis is **plantar fibromatosis (Ledderhose disease)**.

Here's why:

*   **Appearance:** The image shows a thickened, somewhat raised area on the sole of the foot, particularly in the area of the plantar fascia. This is a classic presentation of plantar fibromatosis.
*   **Location:** Plantar fibromatosis typically affects the plantar fascia, which is the thick band of tissue on the bottom of the foot that supports the arch.
*   **Other possibilities:** While other conditions could potentially cause similar changes, the combination of the location and appearance makes plantar fibromatosis the most probable diagnosis.

**Important Considerations:**

*   **This is a preliminary diagnosis based on a single image.** A definitive diagnosis requires a physical examination by a dermatologist, including a thorough assessment of the affected area and potentially a biopsy.
*   **Other conditions:** Other conditions that can cause similar changes include:


In [5]:
from transformers import pipeline
from PIL import Image
import requests
import torch
import gc

# Clear GPU memory before loading pipeline
if 'pipe' in globals():
    del pipe
gc.collect()
torch.cuda.empty_cache()

# Initialize the MedGemma pipeline
pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-4b-it",
    torch_dtype=torch.bfloat16,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

# Load image
image_url = "https://i.ibb.co/ynJDp0zd/EC-33.jpg"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

# Create message with image only
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are an expert dermatologist, diagnose based on te image."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image}
        ]
    }
]

# Generate output
output = pipe(messages, max_new_tokens=200)

# Print result
print(output[0]["generated_text"][-1]["content"])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


Based on the image, the most likely diagnosis is **Erythema Nodosum**.

Here's why:

*   **Appearance:** The image shows a pattern of reddish-brown, raised nodules or plaques on the skin. This is a classic presentation of erythema nodosum.
*   **Location:** Erythema nodosum typically appears on the shins (lower legs), which is consistent with the image.
*   **Other Considerations:** While other conditions could potentially cause similar skin findings, the distribution and appearance are most suggestive of erythema nodosum.

**Important Considerations:**

*   **Differential Diagnosis:** It's crucial to rule out other conditions that can mimic erythema nodosum, such as:
    *   Cellulitis
    *   Erysipelas
    *   Sarcoidosis
    *   Infections (e.g., streptococcal infection)
    *   Drug reactions
    *   Inflammatory bowel disease (


In [6]:
from transformers import pipeline
from PIL import Image
import requests
import torch
import gc

# Clear GPU memory before loading pipeline
if 'pipe' in globals():
    del pipe
gc.collect()
torch.cuda.empty_cache()

# Initialize the MedGemma pipeline
pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-4b-it",
    torch_dtype=torch.bfloat16,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

# Load image
image_url = "https://i.ibb.co/B5MrLM9V/EC-38.jpg"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

# Create message with image only
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are an expert dermatologist, diagnose based on te image."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image}
        ]
    }
]

# Generate output
output = pipe(messages, max_new_tokens=200)

# Print result
print(output[0]["generated_text"][-1]["content"])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


Based on the image, the most likely diagnosis is **Tinea Pedis (Athlete's Foot)**.

Here's why:

*   **Appearance:** The image shows a scaly, erythematous (red) patch with some scaling and possibly some mild inflammation. This is a classic presentation of tinea pedis.
*   **Location:** The lesion is located on the foot, which is a common site for tinea pedis.
*   **Symptoms:** While not explicitly mentioned, tinea pedis often presents with itching, burning, and scaling.

**Important Considerations:**

*   **Differential Diagnosis:** Other possibilities to consider include:
    *   **Eczema:** Eczema can also present with scaling and redness, but it often has a more pruritic (itchy) nature and may be more widespread.
    *   **Psoriasis:** Psoriasis can also cause scaling and redness, but it often has a more silvery


In [7]:
from transformers import pipeline
from PIL import Image
import requests
import torch
import gc

# Clear GPU memory before loading pipeline
if 'pipe' in globals():
    del pipe
gc.collect()
torch.cuda.empty_cache()

# Initialize the MedGemma pipeline
pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-4b-it",
    torch_dtype=torch.bfloat16,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

# Load image
image_url = "https://i.ibb.co/gbJKV7ct/EC-44.jpg"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

# Create message with image only
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are an expert dermatologist, diagnose based on te image."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image}
        ]
    }
]

# Generate output
output = pipe(messages, max_new_tokens=200)

# Print result
print(output[0]["generated_text"][-1]["content"])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


Based on the image, the most likely diagnosis is **acral lentiginous melanoma (ALM)**.

Here's why:

*   **Location:** The lesions are located on the palms and soles, which are common sites for ALM.
*   **Appearance:** The lesions are dark, irregularly shaped, and have a slightly raised surface. This is consistent with the typical presentation of ALM.
*   **Risk Factors:** While not explicitly stated, individuals with darker skin tones are at higher risk for ALM.

It's important to note that this is a preliminary diagnosis based on a single image. A definitive diagnosis requires a clinical examination by a dermatologist, including a biopsy of the lesions to confirm the presence of melanoma cells.

**Disclaimer:** I am an AI and cannot provide medical diagnoses. This information is for educational purposes only and should not be considered a substitute for professional medical advice. Please consult with a qualified dermatologist for any skin concerns.


In [8]:
from transformers import pipeline
from PIL import Image
import requests
import torch
import gc

# Clear GPU memory before loading pipeline
if 'pipe' in globals():
    del pipe
gc.collect()
torch.cuda.empty_cache()

# Initialize the MedGemma pipeline
pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-4b-it",
    torch_dtype=torch.bfloat16,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

# Load image
image_url = "https://i.ibb.co/PsT1RNBY/EC-59.jpg"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

# Create message with image only
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are an expert dermatologist, diagnose based on te image."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image}
        ]
    }
]

# Generate output
output = pipe(messages, max_new_tokens=200)

# Print result
print(output[0]["generated_text"][-1]["content"])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


Based on the image, the most likely diagnosis is **Tinea Pedis (Athlete's Foot)**.

Here's why:

*   **Appearance:** The image shows a scaly, erythematous (red) patch with some scaling and possibly some vesicles (small blisters). This is a classic presentation of tinea pedis.
*   **Location:** The lesion is located on the foot, which is a common site for tinea pedis.
*   **Symptoms:** While I can't definitively say based on the image alone, tinea pedis often presents with itching, burning, and scaling.

**Important Considerations:**

*   **Differential Diagnosis:** Other possibilities could include eczema, psoriasis, or a fungal infection of the nail (onychomycosis).
*   **Further Evaluation:** A definitive diagnosis would require a clinical examination and possibly a skin scraping or fungal culture to confirm the presence of a fungal infection.

**Disclaimer:** This diagnosis is based solely


In [9]:
from transformers import pipeline
from PIL import Image
import requests
import torch
import gc

# Clear GPU memory before loading pipeline
if 'pipe' in globals():
    del pipe
gc.collect()
torch.cuda.empty_cache()

# Initialize the MedGemma pipeline
pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-4b-it",
    torch_dtype=torch.bfloat16,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

# Load image
image_url = "https://i.ibb.co/6c2cmTDZ/EC-74.jpg"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

# Create message with image only
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are an expert dermatologist, diagnose based on te image."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image}
        ]
    }
]

# Generate output
output = pipe(messages, max_new_tokens=200)

# Print result
print(output[0]["generated_text"][-1]["content"])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


Based on the image, I can see a close-up view of the oral mucosa.  It's difficult to provide a definitive diagnosis without a full examination, but I can offer some observations:

*   **Color:** The color appears to be a mix of pink and brown/tan. This could be due to various factors, including blood vessels, pigmentation, or the presence of lesions.
*   **Texture:** The texture appears relatively smooth, but it's hard to assess the fine details without a higher resolution image.
*   **Presence of lesions:** I can see some subtle irregularities, but it's difficult to determine if they are clinically significant.

**Possible considerations:**

*   **Oral lichen planus:** This condition can cause white, red, or purple patches on the oral mucosa.
*   **Oral candidiasis (thrush):** This fungal infection can cause white, curd-like patches.
*   **Oral leukoplakia:** This


In [10]:
from transformers import pipeline
from PIL import Image
import requests
import torch
import gc

# Clear GPU memory before loading pipeline
if 'pipe' in globals():
    del pipe
gc.collect()
torch.cuda.empty_cache()

# Initialize the MedGemma pipeline
pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-4b-it",
    torch_dtype=torch.bfloat16,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

# Load image
image_url = "https://i.ibb.co/FbCKxc1x/EC-91.jpg"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

# Create message with image only
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are an expert dermatologist, diagnose based on te image."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image}
        ]
    }
]

# Generate output
output = pipe(messages, max_new_tokens=200)

# Print result
print(output[0]["generated_text"][-1]["content"])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


Based on the image, the most likely diagnosis is **Lichen Sclerosus**.

Here's why:

*   **Appearance:** The image shows a skin lesion with a white, atrophic (thin and wrinkled) base and a red, raised border. This is a classic presentation of lichen sclerosus.
*   **Location:** Lichen sclerosus commonly affects the genital and perianal areas, but it can also occur on other parts of the body, including the limbs.
*   **Other possibilities:** While other conditions could potentially present with similar features, the combination of the atrophic base and raised border is highly suggestive of lichen sclerosus.

**Important Considerations:**

*   **Differential Diagnosis:** It's crucial to rule out other conditions that can mimic lichen sclerosus, such as:
    *   Lichen planus
    *   Psoriasis
    *   Vitiligo
    *   Other inflammatory skin


In [11]:
from transformers import pipeline
from PIL import Image
import requests
import torch
import gc

# Clear GPU memory before loading pipeline
if 'pipe' in globals():
    del pipe
gc.collect()
torch.cuda.empty_cache()

# Initialize the MedGemma pipeline
pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-4b-it",
    torch_dtype=torch.bfloat16,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

# Load image
image_url = "https://i.ibb.co/xSkFjjj4/EC-105.jpg"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

# Create message with image only
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are an expert dermatologist, diagnose based on te image."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image}
        ]
    }
]

# Generate output
output = pipe(messages, max_new_tokens=200)

# Print result
print(output[0]["generated_text"][-1]["content"])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


Based on the image, the most likely diagnosis is **Tinea Manuum (Ringworm of the Hand)**.

Here's why:

*   **Appearance:** The image shows a circular, raised, scaly, and itchy rash on the hand. This is a classic presentation of tinea manuum.
*   **Location:** The rash is localized to the hand, which is consistent with tinea manuum.
*   **Symptoms:** The description of the rash as itchy is also typical of tinea infections.

**Other possibilities, though less likely, could include:**

*   **Contact Dermatitis:** This could present with a similar rash, but it's often more localized to the area of contact with an irritant or allergen.
*   **Eczema:** Eczema can cause itchy, scaly patches, but it often presents with more widespread involvement and may have a different appearance.
*   **Psoriasis:** Psoriasis


In [12]:
from transformers import pipeline
from PIL import Image
import requests
import torch
import gc

# Clear GPU memory before loading pipeline
if 'pipe' in globals():
    del pipe
gc.collect()
torch.cuda.empty_cache()

# Initialize the MedGemma pipeline
pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-4b-it",
    torch_dtype=torch.bfloat16,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

# Load image
image_url = "https://i.ibb.co/XZJYFYMV/EC-108.jpg"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

# Create message with image only
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are an expert dermatologist, diagnose based on te image."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image}
        ]
    }
]

# Generate output
output = pipe(messages, max_new_tokens=200)

# Print result
print(output[0]["generated_text"][-1]["content"])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


Based on the image, the most likely diagnosis is a **basal cell carcinoma (BCC)**.

Here's why:

*   **Appearance:** The lesion is a pearly or waxy bump with visible blood vessels (telangiectasias) on the surface. This is a classic presentation of BCC.
*   **Location:** BCCs are commonly found on sun-exposed areas, such as the trunk, face, and ears.
*   **Other possibilities:** While other skin conditions can present with similar features, the combination of the pearly bump, telangiectasias, and location makes BCC the most probable diagnosis.

**Important Considerations:**

*   **This is a preliminary diagnosis based on a single image.** A definitive diagnosis requires a clinical examination by a dermatologist, potentially including a biopsy.
*   **Other conditions:** Other possibilities to consider include squamous cell carcinoma (SCC) and actinic keratosis, but BCC is the most common.


In [1]:
from transformers import pipeline
from PIL import Image
import requests
import torch
import gc

# Clear GPU memory before loading pipeline
if 'pipe' in globals():
    del pipe
gc.collect()
torch.cuda.empty_cache()

# Initialize the MedGemma pipeline
pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-4b-it",
    torch_dtype=torch.bfloat16,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

# Load image
image_url = "https://i.ibb.co/Mxz7GqdJ/EC-119.jpg"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

# Create message with image only
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": " As an expert in dermatologist, Describe the findings and most likely diagnose in this image. Be concise"}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image}
        ]
    }
]

# Generate output
output = pipe(messages, max_new_tokens=200)

# Print result
print(output[0]["generated_text"][-1]["content"])

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cuda


Based on the image, the most likely diagnosis is a **basal cell carcinoma (BCC)**.

**Findings:**

*   **Erythematous, pearly or waxy bump:** This is a classic presentation of BCC.
*   **Telangiectasias (small, visible blood vessels):** These are often present on the surface of BCCs.
*   **Possible ulceration:** The image shows some signs of ulceration, which can occur in advanced BCCs.

**Important Note:** This is a preliminary assessment based on a single image. A definitive diagnosis requires a clinical examination and potentially a biopsy.



In [2]:
from transformers import pipeline
from PIL import Image
import requests
import torch
import gc

# Clear GPU memory before loading pipeline
if 'pipe' in globals():
    del pipe
gc.collect()
torch.cuda.empty_cache()

# Initialize the MedGemma pipeline
pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-4b-it",
    torch_dtype=torch.bfloat16,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

# Load image
image_url = "https://i.ibb.co/pjzZ63wn/EC-133.jpg"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

# Create message with image only
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are an expert dermatologist, diagnose based on te image."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image}
        ]
    }
]

# Generate output
output = pipe(messages, max_new_tokens=200)

# Print result
print(output[0]["generated_text"][-1]["content"])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


Based on the image, the most likely diagnosis is **skin tag**.

Here's why:

*   **Appearance:** Skin tags are typically flesh-colored or slightly darker, soft, and pedunculated (attached by a stalk). They are often found in areas of friction, such as the armpits, groin, and neck.
*   **Location:** The image shows a skin tag on the body.

However, it's important to note that I am an AI and cannot provide a definitive diagnosis. A proper diagnosis requires a physical examination by a qualified dermatologist.

**Recommendations:**

1.  **Consult a Dermatologist:** The best course of action is to see a dermatologist for a proper examination and diagnosis. They can perform a skin biopsy if necessary to rule out other conditions.
2.  **Avoid Picking or Scratching:** Do not attempt to remove the skin tag yourself, as this can lead to infection or scarring.
3.  **Consider Professional Removal


In [3]:
from transformers import pipeline
from PIL import Image
import requests
import torch
import gc

# Clear GPU memory before loading pipeline
if 'pipe' in globals():
    del pipe
gc.collect()
torch.cuda.empty_cache()

# Initialize the MedGemma pipeline
pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-4b-it",
    torch_dtype=torch.bfloat16,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

# Load image
image_url = "https://i.ibb.co/1Yg3dZyr/EC-145.jpg"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

# Create message with image only
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are an expert dermatologist, diagnose based on te image."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image}
        ]
    }
]

# Generate output
output = pipe(messages, max_new_tokens=200)

# Print result
print(output[0]["generated_text"][-1]["content"])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


Based on the image, the most likely diagnosis is **Tinea pedis**, commonly known as athlete's foot.

Here's why:

*   **Appearance:** The image shows scaling, redness, and possibly some pinpoint black dots (miliaria) on the soles of the feet. These are classic signs of tinea pedis.
*   **Location:** The condition is localized to the feet, which is typical for athlete's foot.
*   **Other possibilities:** While other conditions could potentially cause similar symptoms, tinea pedis is the most common and fits the description best.

**Important Considerations:**

*   This is a preliminary diagnosis based on a single image. A definitive diagnosis requires a clinical examination, including a detailed history of the patient's symptoms and potential risk factors.
*   Other conditions, such as eczema, psoriasis, or fungal infections of the nails, could also present with similar skin changes.
*   It is important to


In [4]:
from transformers import pipeline
from PIL import Image
import requests
import torch
import gc

# Clear GPU memory before loading pipeline
if 'pipe' in globals():
    del pipe
gc.collect()
torch.cuda.empty_cache()

# Initialize the MedGemma pipeline
pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-4b-it",
    torch_dtype=torch.bfloat16,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

# Load image
image_url = "https://i.ibb.co/RkyYdGkR/EC-196.jpg"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

# Create message with image only
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are an expert dermatologist, diagnose based on te image."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image}
        ]
    }
]

# Generate output
output = pipe(messages, max_new_tokens=200)

# Print result
print(output[0]["generated_text"][-1]["content"])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


Based on the image, the most likely diagnosis is a **keratosis pilaris**.

Here's why:

*   **Appearance:** The image shows small, raised, rough bumps on the skin. These are characteristic of keratosis pilaris.
*   **Location:** Keratosis pilaris commonly appears on the upper arms, thighs, cheeks, and buttocks.
*   **Texture:** The texture of the bumps is often described as "chicken skin."

**Important Considerations:**

*   This is a preliminary diagnosis based on a single image. A definitive diagnosis requires a physical examination by a dermatologist.
*   Other conditions, such as folliculitis or eczema, can sometimes mimic keratosis pilaris.
*   If you are concerned about the appearance of a skin lesion, it is always best to consult a dermatologist for a proper diagnosis and treatment plan.

**Disclaimer:** I am an AI and cannot provide medical advice. This information is for educational


In [5]:
from transformers import pipeline
from PIL import Image
import requests
import torch
import gc

# Clear GPU memory before loading pipeline
if 'pipe' in globals():
    del pipe
gc.collect()
torch.cuda.empty_cache()

# Initialize the MedGemma pipeline
pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-4b-it",
    torch_dtype=torch.bfloat16,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

# Load image
image_url = "https://i.ibb.co/cK7CZKk4/EC-221.jpg"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

# Create message with image only
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": " As an expert in dermatologist, Describe the findings and most likely diagnose in this image. Be concise"}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image}
        ]
    }
]

# Generate output
output = pipe(messages, max_new_tokens=200)

# Print result
print(output[0]["generated_text"][-1]["content"])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


Based on the image, the most likely diagnosis is **keratosis pilaris**.

**Findings:**

*   Small, rough, and slightly raised bumps, often described as "chicken skin."
*   Typically found on the upper arms, thighs, cheeks, or buttocks.

**Differential Diagnosis:**

*   Other conditions with similar appearances could include eczema, psoriasis, or other skin conditions.

**Important Note:** This is a preliminary assessment based on a single image. A definitive diagnosis requires a clinical examination and potentially a skin biopsy.



In [6]:
from transformers import pipeline
from PIL import Image
import requests
import torch
import gc

# Clear GPU memory before loading pipeline
if 'pipe' in globals():
    del pipe
gc.collect()
torch.cuda.empty_cache()

# Initialize the MedGemma pipeline
pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-4b-it",
    torch_dtype=torch.bfloat16,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

# Load image
image_url = "https://i.ibb.co/KzK2cDN2/EC-361.jpg"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

# Create message with image only
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are an expert dermatologist, diagnose based on te image."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image}
        ]
    }
]

# Generate output
output = pipe(messages, max_new_tokens=200)

# Print result
print(output[0]["generated_text"][-1]["content"])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


Based on the image, I can see a lesion with a slightly raised, irregular border and a darker central area. It's difficult to provide a definitive diagnosis without a clinical examination and potentially a biopsy. However, based on the appearance, I would recommend the following:

**Possible Diagnoses (Differential Diagnosis):**

*   **Actinic Keratosis (Solar Keratosis):** This is a common precancerous skin lesion caused by sun exposure. It often appears as a rough, scaly patch.
*   **Seborrheic Keratosis:** These are benign skin growths that are common in older adults. They can be waxy, scaly, or crusty.
*   **Basal Cell Carcinoma (BCC):** This is the most common type of skin cancer. It often appears as a pearly or waxy bump, a flat, flesh-colored or brown scar-like lesion, or a sore that bleeds and heals, then recurs.


In [ ]:
from transformers import pipeline
from PIL import Image
import requests
import torch
import gc

# Clear GPU memory before loading pipeline
if 'pipe' in globals():
    del pipe
gc.collect()
torch.cuda.empty_cache()

# Initialize the MedGemma pipeline
pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-4b-it",
    torch_dtype=torch.bfloat16,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

# Load image
image_url = "https://i.ibb.co/CsbtWCJV/EC-394.jpg"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

# Create message with image only
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are an expert dermatologist, diagnose based on te image."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image}
        ]
    }
]

# Generate output
output = pipe(messages, max_new_tokens=200)

# Print result
print(output[0]["generated_text"][-1]["content"])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


In [ ]:
from transformers import pipeline
from PIL import Image
import requests
import torch
import gc

# Clear GPU memory before loading pipeline
if 'pipe' in globals():
    del pipe
gc.collect()
torch.cuda.empty_cache()

# Initialize the MedGemma pipeline
pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-4b-it",
    torch_dtype=torch.bfloat16,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

# Load image
image_url = "https://i.ibb.co/Mby4Grh/EC-130.jpg"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

# Create message with image only
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are an expert dermatologist, diagnose based on te image."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image}
        ]
    }
]

# Generate output
output = pipe(messages, max_new_tokens=200)

# Print result
print(output[0]["generated_text"][-1]["content"])

In [ ]:
from transformers import pipeline
from PIL import Image
import requests
import torch
import gc

# Clear GPU memory before loading pipeline
if 'pipe' in globals():
    del pipe
gc.collect()
torch.cuda.empty_cache()

# Initialize the MedGemma pipeline
pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-4b-it",
    torch_dtype=torch.bfloat16,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

# Load image
image_url = "https://i.ibb.co/dsZMJjmJ/EC-117.jpg"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

# Create message with image only
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are an expert dermatologist, diagnose based on te image."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image}
        ]
    }
]

# Generate output
output = pipe(messages, max_new_tokens=200)

# Print result
print(output[0]["generated_text"][-1]["content"])

In [ ]:
from transformers import pipeline
from PIL import Image
import requests
import torch
import gc

# Clear GPU memory before loading pipeline
if 'pipe' in globals():
    del pipe
gc.collect()
torch.cuda.empty_cache()

# Initialize the MedGemma pipeline
pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-4b-it",
    torch_dtype=torch.bfloat16,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

# Load image
image_url = "https://i.ibb.co/LhRXKgZy/EC-95.jpg"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

# Create message with image only
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are an expert dermatologist, diagnose based on te image."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image}
        ]
    }
]

# Generate output
output = pipe(messages, max_new_tokens=200)

# Print result
print(output[0]["generated_text"][-1]["content"])

In [ ]:
from transformers import pipeline
from PIL import Image
import requests
import torch
import gc

# Clear GPU memory before loading pipeline
if 'pipe' in globals():
    del pipe
gc.collect()
torch.cuda.empty_cache()

# Initialize the MedGemma pipeline
pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-4b-it",
    torch_dtype=torch.bfloat16,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

# Load image
image_url = "https://i.ibb.co/WWZbLn8v/EC-57.jpg"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

# Create message with image only
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are an expert dermatologist, diagnose based on te image."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image}
        ]
    }
]

# Generate output
output = pipe(messages, max_new_tokens=200)

# Print result
print(output[0]["generated_text"][-1]["content"])

In [ ]:
from transformers import pipeline
from PIL import Image
import requests
import torch
import gc

# Clear GPU memory before loading pipeline
if 'pipe' in globals():
    del pipe
gc.collect()
torch.cuda.empty_cache()

# Initialize the MedGemma pipeline
pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-4b-it",
    torch_dtype=torch.bfloat16,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

# Load image
image_url = "https://i.ibb.co/gbMFhcRt/EC-66.jpg"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

# Create message with image only
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are an expert dermatologist, diagnose based on te image."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image}
        ]
    }
]

# Generate output
output = pipe(messages, max_new_tokens=200)

# Print result
print(output[0]["generated_text"][-1]["content"])

In [ ]:
from transformers import pipeline
from PIL import Image
import requests
import torch
import gc

# Clear GPU memory before loading pipeline
if 'pipe' in globals():
    del pipe
gc.collect()
torch.cuda.empty_cache()

# Initialize the MedGemma pipeline
pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-4b-it",
    torch_dtype=torch.bfloat16,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

# Load image
image_url = "https://i.ibb.co/gbMFhcRt/EC-51.jpg"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

# Create message with image only
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are an expert dermatologist, diagnose based on te image."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image}
        ]
    }
]

# Generate output
output = pipe(messages, max_new_tokens=200)

# Print result
print(output[0]["generated_text"][-1]["content"])

In [ ]:
from transformers import pipeline
from PIL import Image
import requests
import torch
import gc

# Clear GPU memory before loading pipeline
if 'pipe' in globals():
    del pipe
gc.collect()
torch.cuda.empty_cache()

# Initialize the MedGemma pipeline
pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-4b-it",
    torch_dtype=torch.bfloat16,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

# Load image
image_url = "https://i.ibb.co/gbMFhcRt/EC-30.jpg"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

# Create message with image only
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are an expert dermatologist, diagnose based on te image."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image}
        ]
    }
]

# Generate output
output = pipe(messages, max_new_tokens=200)

# Print result
print(output[0]["generated_text"][-1]["content"])

In [ ]:
from transformers import pipeline
from PIL import Image
import requests
import torch
import gc

# Clear GPU memory before loading pipeline
if 'pipe' in globals():
    del pipe
gc.collect()
torch.cuda.empty_cache()

# Initialize the MedGemma pipeline
pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-4b-it",
    torch_dtype=torch.bfloat16,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

# Load image
image_url = "https://i.ibb.co/gbMFhcRt/EC-20.jpg"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

# Create message with image only
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are an expert dermatologist, diagnose based on te image."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image}
        ]
    }
]

# Generate output
output = pipe(messages, max_new_tokens=200)

# Print result
print(output[0]["generated_text"][-1]["content"])

In [ ]:
from transformers import pipeline
from PIL import Image
import requests
import torch
import gc

# Clear GPU memory before loading pipeline
if 'pipe' in globals():
    del pipe
gc.collect()
torch.cuda.empty_cache()

# Initialize the MedGemma pipeline
pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-4b-it",
    torch_dtype=torch.bfloat16,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

# Load image
image_url = "https://i.ibb.co/gbMFhcRt/EC-16.jpg"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

# Create message with image only
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are an expert dermatologist, diagnose based on te image."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image}
        ]
    }
]

# Generate output
output = pipe(messages, max_new_tokens=200)

# Print result
print(output[0]["generated_text"][-1]["content"])